In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install transformers datasets

In [3]:
import pandas as pd
import numpy as np
import os

import nltk
nltk.download('punkt')

# from datasets import load_dataset
# import datasets
# from datasets import Dataset

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Import the data

In [4]:
# df_idiomatic_en = pd.read_csv("../Data/data_en_idio.csv")
# df_non_idiomatic_en = pd.read_csv("../Data/data_en_non_idio.csv")

# df_idiomatic_pt = pd.read_csv("../Data/data_pt_idio.csv")
# df_non_idiomatic_pt = pd.read_csv("../Data/data_pt_non_idio.csv")

# df_idiomatic_gl = pd.read_csv("../Data/data_gl_idio.csv")
# df_non_idiomatic_gl = pd.read_csv("../Data/data_gl_non_idio.csv")

df_idiomatic_en = pd.read_csv("../Data/data_1_semeval_all.csv")

In [5]:
# Previous + Target
df_idiomatic_en['Context'] = df_idiomatic_en['Previous'] + df_idiomatic_en['Target']
df_idiomatic_en['All'] = df_idiomatic_en['Previous'] + df_idiomatic_en['Target'] + df_idiomatic_en['Next']
df_idiomatic_en.head()

,Unnamed: 0,MWE,Previous,Target,Next,Context,All
0,0,double dutch,This inspired others to jump ropes as a leisur...,There are several theories behind the origin o...,The most popular theory states that “Double Du...,This inspired others to jump ropes as a leisur...,This inspired others to jump ropes as a leisur...
1,1,double dutch,In the age of chivalry a man paid for the woma...,"Double Dutch also derives from the same era, D...",There are many phrases that include the word: ...,In the age of chivalry a man paid for the woma...,In the age of chivalry a man paid for the woma...
2,2,double dutch,"To her eternal credit, she kept both India and...",Since 1977 we have had a plethora of Foreign M...,We need to exclude from that list the late Mr ...,"To her eternal credit, she kept both India and...","To her eternal credit, she kept both India and..."
3,3,double dutch,While pharmaceutical companies were researchin...,Turns out that these people were speaking doub...,So why aren’t Big Macs sold all over the world...,While pharmaceutical companies were researchin...,While pharmaceutical companies were researchin...
4,4,double dutch,Coronavirus in Europe * Brexit * Brussels ...,Is Flemish premier talking double Dutch?,Three months before the Belgians take over the...,Coronavirus in Europe * Brexit * Brussels ...,Coronavirus in Europe * Brexit * Brussels ...


In [6]:
# drop the rows with nan values

nan_rows = df_idiomatic_en.loc[pd.isna(df_idiomatic_en["All"]), :].index
df_idiomatic_en = df_idiomatic_en.drop([df_idiomatic_en.index[nan_rows[0]], df_idiomatic_en.index[nan_rows[1]]])

In [7]:
from statistics import mean

def data_stats(df):
    l = list(df.apply(len))
    print("Max lentgh:", max(l))
    print(df.iloc[l.index(max(l))])
    print("\nSmallest length:", min(l))
    print(df.iloc[l.index(min(l))])
    print("\nAvg length:", mean(l))

In [8]:
data_stats(df_idiomatic_en['Previous'])

Max lentgh: 1231
I'm sorry for everything I never did   I'm sorry about all the things I've never said   I'm sorry I've been such a hassle for you all to string along   I'm sorry that I'm not proud   I'm sorry that I feel like I belong in the ground   I'm sorry the concrete feels more like home   I'm sorry that everyday I feel more alone      Just one of those days      I’m sorry that every second I feel more weak   I'm sorry I'm caught up in all that I'll never be   I’m sorry that I always talk too much   I’m sorry that nothing was good enough   I'm sorry that I’m a has-been, with his smile dimmed   Just a nuisance who can't take a hint   I'm sorry that I could never help   I'm sorry for feeling sorry for myself      Just one of those days where I just want you to stay      I swear to god I tried my best   But the world won’t stop beating on my chest   I swear that it’s cracked my skull   And made everything I feel so dull   I swear to god it’s not your fault   I just never learned ho

In [9]:
# split into train and test
from sklearn.model_selection import train_test_split

df_train_en, df_test_en = train_test_split(df_idiomatic_en, test_size=0.2, random_state=42)

## Models 

In [10]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.4 MB/s eta 0:00:00


In [11]:
from transformers import pipeline, set_seed

In [12]:
# df_idiomatic_en['Previous + Target'] = df_idiomatic_en['Previous'] + df_idiomatic_en['Target']
# # df_idiomatic_en.head()

###GPT1

In [13]:
def gpt1_generator(df):
    next_sentences = []
    for i in df:

        generator = pipeline('text-generation', model='openai-gpt')
        set_seed(42)
        # s3 = generator(i, max_length=15, num_return_sequences=1)  # max_length defaults to 20
        s3 = generator(i, max_new_tokens=15, num_return_sequences=1)
        s3 = s3[0]['generated_text'][len(i):]
        
        # everything before the first period
        next_sentences.append(s3.split('.')[0])

    return next_sentences

In [ ]:
next_sent = gpt1_generator(df_test_en['Context'])

Downloading:   0%|          | 0.00/656 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479M [00:00<?, ?B/s]

Some weights of OpenAIGPTLMHeadModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/816k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/458k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Some weights of OpenAIGPTLMHeadModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of OpenAIGPTLMHeadModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of OpenAIGPTLMHeadModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of OpenAIGPTLMHeadModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and

In [ ]:
df_test_en['Generated Next'] = next_sent
df_test_en.head()

,Unnamed: 0,MWE,Previous,Target,Next,Context,All,Generated Next
1962,452,bad hat,And you are the marvellous exception.,"Lovely as a goddess, clever as an Athenian and...",I’m going to kiss the lips off you once we lan...,And you are the marvellous exception.Lovely as...,And you are the marvellous exception.Lovely as...,\n the one who was not very smart?'\n i laughed
527,963,beauty sleep,He apparently notified his side and made them ...,"Well, we were in bed getting our beauty sleep ...",There was no need for a rush or anything.,He apparently notified his side and made them ...,He apparently notified his side and made them ...,"\n the night before was much like last time, ..."
393,694,grandfather clock,If you are interested in a grandfather clock t...,This Howard Miller grandfather clock provides ...,This has all the look and feel of a grandfathe...,If you are interested in a grandfather clock t...,If you are interested in a grandfather clock t...,\n the one that was installed was in the grea...
1404,2708,dark horse,American Airlines adds flights from DFW to Mex...,Breaking down Texas’ 2021 recruiting class: Th...,Texas’ 2021 class ranked 17th on 247Sports Com...,American Airlines adds flights from DFW to Mex...,American Airlines adds flights from DFW to Mex...,", the résumés from the national agencies that ..."
433,797,contact lens,Half or more of wearers reported ever sleeping...,Approximately one third (35.5%) of contact len...,Almost all rigid wearers (91.3%) reported ever...,Half or more of wearers reported ever sleeping...,Half or more of wearers reported ever sleeping...,\n the one hundred was much easier


###GPT2

In [ ]:
def gpt2_generator(df):
    next_sentences = []
    for i in df[:5]:

        generator = pipeline('text-generation', model='gpt2')
        set_seed(42)
        s3 = generator(i, max_new_tokens=15, num_return_sequences=1)
        s3 = s3[0]['generated_text'][len(i):]
        
        # # everything before the first period
        # next_sentences.append(s3.split('.')[0])
        next_sentences.append(s3)

    return next_sentences

In [ ]:
next_sent = gpt2_generator(df_test_en['Context'])

In [ ]:
df_test_en['Generated Next GPT2'] = next_sent
df_test_en.head()